In [ ]:
# Core
import pandas as pd, numpy as np, matplotlib.pyplot as plt, tensorflow as tf
from tensorflow.keras import layers, models, callbacks, losses, mixed_precision

# Scikit-learn helpers
from sklearn.compose        import ColumnTransformer
from sklearn.preprocessing   import OneHotEncoder, StandardScaler
from sklearn.decomposition   import TruncatedSVD
from sklearn.ensemble        import IsolationForest
from sklearn.metrics         import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, ConfusionMatrixDisplay
)
from sklearn.model_selection import train_test_split

# --- Reproducibility --------------------------------------------------------
RANDOM_STATE = 42
tf.random.set_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

# --- SPEED + MEMORY TUNING --------------------------------------------------
mixed_precision.set_global_policy("mixed_float16")   # fast *and* half-sized tensors
# tf.config.optimizer.set_jit(True)   # comment out or delete this line

BATCH      = 1024        # 1 k fits easily; raise or lower as needed
CACHE_TRAIN = "./ae_train.cache"   # on-disk cache files
CACHE_VAL   = "./ae_val.cache"


In [ ]:
# Helper: load & prepare a CSV (+labels)
def load_and_prepare(path_csv, path_labels=None):
    """
    Load a transactions CSV and optional labels JSON.
    Adds hour_of_day & day_of_week. Supports either
    string or numeric amount.
    """
    df = pd.read_csv(
        path_csv,
        usecols=["id", "date", "amount", "merchant_city", "mcc"]
    )
    df["date"] = pd.to_datetime(df["date"])

    # robust amount cleaning
    if pd.api.types.is_numeric_dtype(df["amount"]):
        df["amount"] = df["amount"].astype(float)
    else:
        df["amount"] = (
            df["amount"]
              .astype(str)
              .str.replace(r"[$,]", "", regex=True)
              .astype(float)
        )

    df = df.rename(columns={"id": "transaction_id",
                            "merchant_city": "location"})
    df["hour_of_day"] = df["date"].dt.hour
    df["day_of_week"] = df["date"].dt.weekday

    if path_labels is not None:
        lbl = (
            pd.read_json(path_labels)
              .reset_index()
              .rename(columns={"index": "transaction_id"})
              .replace({"target": {"Yes": 1, "No": 0}})
        )
        df = df.merge(lbl, on="transaction_id")

    # inject target column if it exists
    df_target = pd.read_csv(path_csv)
    if "target" in df_target.columns:
        df["target"] = df_target["target"].astype(int)

    return df

In [ ]:
# ColumnTransformer + SVD reducer
cat_cols = ["location", "mcc"]
num_cols = ["amount", "hour_of_day", "day_of_week"]

base_pre = ColumnTransformer(
    transformers=[
        ("cat",
         OneHotEncoder(handle_unknown="ignore",
                       sparse_output=True,
                       dtype=np.int8),
         cat_cols),
        ("num", StandardScaler(), num_cols)
    ],
    sparse_threshold=0.3
)

# Reduce high-dim sparse matrix to 200 dense features
svd = TruncatedSVD(
        n_components=200,
        algorithm="randomized",
        n_iter=5,
        random_state=RANDOM_STATE
)


In [ ]:
# Train Isolation Forest on Kaggle
kag_csv   = "../assets/kaggle_data/transactions_data.csv"
kag_lbl   = "../assets/kaggle_data/train_fraud_labels.json"

train_df  = load_and_prepare(kag_csv, kag_lbl)
y_train   = train_df["target"]
X_train   = train_df.drop(columns=["transaction_id", "date", "target"])

# 1️⃣  sparse one-hot/scaled  →  2️⃣  SVD  →  dense 200-col
X_train_sparse   = base_pre.fit_transform(X_train)
scaler = StandardScaler()
X_train_reduced  = svd.fit_transform(X_train_sparse)
X_train_scaled   = scaler.fit_transform(X_train_reduced)

iso = IsolationForest(
        n_estimators=300,
        contamination=0.10,         # trim 10 % of the data
        max_samples="auto",
        n_jobs=-1,
        random_state=RANDOM_STATE
)
iso.fit(X_train_scaled)

# Keep only normal points for AE training
mask_keep   = iso.predict(X_train_scaled) == 1
X_ae_dense  = scaler.transform(X_train_reduced[mask_keep]).astype("float32")
print("AE training set shape:", X_ae_dense.shape)


In [ ]:
# ---------------------------------------------------------------------------
# Build train / validation datasets  (FP16 + on-disk cache)
# ---------------------------------------------------------------------------
X_train, X_val = train_test_split(
    X_ae_dense, test_size=0.20, random_state=RANDOM_STATE)

AUTOTUNE = tf.data.AUTOTUNE

def make_ds(X, cache_path):
    X = X.astype("float16")                       # ½ the size of FP32
    ds = tf.data.Dataset.from_tensor_slices((X, X))
    ds = ds.cache(cache_path)                     # on-disk, not RAM
    ds = ds.shuffle(buffer_size=min(20_000, len(X)),
                    seed=RANDOM_STATE)
    ds = ds.batch(BATCH)                          # no drop_remainder → smaller tail batch
    return ds.prefetch(AUTOTUNE)

ds_train = make_ds(X_train, CACHE_TRAIN)
ds_val   = make_ds(X_val,   CACHE_VAL)

# ---------------------------------------------------------------------------
# Auto-encoder architecture  (unchanged)
# ---------------------------------------------------------------------------
n_in = X_ae_dense.shape[1]
enc_units      = [256, 128, 64, 32]
bottleneck_dim = 16
dec_units      = [32, 64, 128, 256]

inp = layers.Input(shape=(n_in,))
x   = layers.GaussianNoise(0.02)(inp)

for u in enc_units:
    x = layers.Dense(u, kernel_regularizer=tf.keras.regularizers.l2(1e-5))(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.1)(x)
    x = layers.Dropout(0.2)(x)

x = layers.Dense(bottleneck_dim,
                 kernel_regularizer=tf.keras.regularizers.l2(1e-5))(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(alpha=0.1)(x)

for u in dec_units:
    x = layers.Dense(u, kernel_regularizer=tf.keras.regularizers.l2(1e-5))(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.1)(x)

out = layers.Dense(n_in, activation="linear", dtype="float32")(x)

ae = models.Model(inp, out)

lr_sched = tf.keras.optimizers.schedules.ExponentialDecay(
    1e-3, decay_steps=2_000, decay_rate=0.90)
opt = tf.keras.optimizers.AdamW(learning_rate=lr_sched, weight_decay=1e-5)

ae.compile(optimizer=opt,
           loss=losses.Huber(delta=1.0),
           run_eagerly=False)

early_stop = callbacks.EarlyStopping(
    patience=12, restore_best_weights=True, verbose=1)

history = ae.fit(
    ds_train,
    epochs=300,
    validation_data=ds_val,
    callbacks=[early_stop],
    verbose=2
)


In [ ]:
# Compute AE threshold (99.99 percentile)
val_size  = int(0.1 * len(X_ae_dense))
recon_val = ae.predict(X_ae_dense[:val_size], batch_size=512, verbose=0)
err_val   = np.mean(np.square(recon_val - X_ae_dense[:val_size]), axis=1)

THRESH = np.percentile(err_val, 99.99)
print(f"Reconstruction-error threshold = {THRESH:.6f}")


In [ ]:
# Predict on chatgpt_data set
test_csv = "assets/chatgpt_data/transactions_o3_2_test.csv"
test_lbl = None   # set path if you have labels

test_df   = load_and_prepare(test_csv, test_lbl)
has_label = "target" in test_df.columns
if has_label:
    y_test = test_df["target"]

X_test = test_df.drop(columns=["transaction_id", "date"] +
                                (["target"] if has_label else []))

# Stage 1 — IF
X_test_sparse   = base_pre.transform(X_test)
X_test_reduced  = svd.transform(X_test_sparse)
X_test_scaled   = scaler.transform(X_test_reduced)

if_flag = (iso.predict(X_test_scaled) == -1).astype(int)

# Stage 2 — AE
recon_test = ae.predict(X_test_scaled.astype("float32"),
                        batch_size=1024, verbose=0)
err_test   = np.mean(np.square(recon_test - X_test_scaled), axis=1)
ae_flag    = (err_test > THRESH).astype(int)

y_pred = (if_flag | ae_flag)


In [ ]:
# Metrics
if has_label:
    print("Accuracy :",  accuracy_score(y_test, y_pred))
    print("Precision:",  precision_score(y_test, y_pred, zero_division=0))
    print("Recall   :",  recall_score(y_test, y_pred))
    print("F1       :",  f1_score(y_test, y_pred))

    cm   = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(cm, display_labels=["No Fraud", "Fraud"])
    disp.plot(cmap=plt.cm.Blues)
    plt.title("Confusion Matrix — chatgpt_data")
    plt.tight_layout()
    plt.show()
else:
    print("Predictions generated (no ground-truth labels supplied).")